# Spotify

## Setup

In [37]:
from pyspark.sql import SparkSession
import pandas as ps
from pyspark.sql import functions as F
from pyspark.sql.functions import col,isnan, when, count
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

In [41]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder

In [5]:
spark = SparkSession.builder.appName('abc').getOrCreate()

In [43]:
sdf = spark.read.format("csv").option("header", "true").option(
    "inferSchema", "true").load("./data/Spotify-2000.csv")
sdf.show()

+-----+--------------------+-----------------+--------------------+----+----------------------+------+------------+-------------+--------+-------+-----------------+------------+-----------+----------+
|Index|               Title|           Artist|           Top Genre|Year|Beats Per Minute (BPM)|Energy|Danceability|Loudness (dB)|Liveness|Valence|Length (Duration)|Acousticness|Speechiness|Popularity|
+-----+--------------------+-----------------+--------------------+----+----------------------+------+------------+-------------+--------+-------+-----------------+------------+-----------+----------+
|    1|             Sunrise|      Norah Jones|     adult standards|2004|                   157|    30|          53|          -14|      11|     68|              201|          94|          3|        71|
|    2|         Black Night|      Deep Purple|          album rock|2000|                   135|    79|          50|          -11|      17|     81|              207|          17|          7|       

In [46]:
sdf.cache()

DataFrame[Index: int, Title: string, Artist: string, Top Genre: string, Year: int, Beats Per Minute (BPM): int, Energy: int, Danceability: int, Loudness (dB): int, Liveness: int, Valence: int, Length (Duration): string, Acousticness: int, Speechiness: int, Popularity: int]

In [47]:
sdf.is_cached

True

In [48]:
sdf.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Top Genre: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Beats Per Minute (BPM): integer (nullable = true)
 |-- Energy: integer (nullable = true)
 |-- Danceability: integer (nullable = true)
 |-- Loudness (dB): integer (nullable = true)
 |-- Liveness: integer (nullable = true)
 |-- Valence: integer (nullable = true)
 |-- Length (Duration): string (nullable = true)
 |-- Acousticness: integer (nullable = true)
 |-- Speechiness: integer (nullable = true)
 |-- Popularity: integer (nullable = true)



## Data exploration

In [30]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+-----+------+---------+----+----------------------+------+------------+-------------+--------+-------+-----------------+------------+-----------+----------+
|Index|Title|Artist|Top Genre|Year|Beats Per Minute (BPM)|Energy|Danceability|Loudness (dB)|Liveness|Valence|Length (Duration)|Acousticness|Speechiness|Popularity|
+-----+-----+------+---------+----+----------------------+------+------------+-------------+--------+-------+-----------------+------------+-----------+----------+
|    0|    0|     0|        0|   0|                     0|     0|           0|            0|       0|      0|                0|           0|          0|         0|
+-----+-----+------+---------+----+----------------------+------+------------+-------------+--------+-------+-----------------+------------+-----------+----------+



## Describe

In [56]:
sdf.describe().toPandas()  # # par défaut ttes les col 
sdf.dtypes
col_num = sdf.select([col[0] for col in sdf.dtypes if col[1] != 'string'])
col_num.show()

+-----+----+----------------------+------+------------+-------------+--------+-------+------------+-----------+----------+
|Index|Year|Beats Per Minute (BPM)|Energy|Danceability|Loudness (dB)|Liveness|Valence|Acousticness|Speechiness|Popularity|
+-----+----+----------------------+------+------------+-------------+--------+-------+------------+-----------+----------+
|    1|2004|                   157|    30|          53|          -14|      11|     68|          94|          3|        71|
|    2|2000|                   135|    79|          50|          -11|      17|     81|          17|          7|        39|
|    3|2001|                   168|    69|          66|           -9|       7|     52|           2|         17|        69|
|    4|2007|                   173|    96|          43|           -4|       3|     37|           0|          4|        76|
|    5|2002|                   106|    82|          58|           -5|      10|     87|           1|          3|        59|
|    6|2004|    

In [57]:
sdf.describe().toPandas()  # # par défaut ttes les col 
sdf.dtypes
col_cat = sdf.select([col[0] for col in sdf.dtypes if col[1] == 'string'])
col_cat.show()

+--------------------+-----------------+--------------------+-----------------+
|               Title|           Artist|           Top Genre|Length (Duration)|
+--------------------+-----------------+--------------------+-----------------+
|             Sunrise|      Norah Jones|     adult standards|              201|
|         Black Night|      Deep Purple|          album rock|              207|
|      Clint Eastwood|         Gorillaz| alternative hip hop|              341|
|       The Pretender|     Foo Fighters|   alternative metal|              269|
|Waitin' On A Sunn...|Bruce Springsteen|        classic rock|              256|
|The Road Ahead (M...|     City To City|alternative pop rock|              247|
|   She Will Be Loved|         Maroon 5|                 pop|              257|
|  Knights of Cydonia|             Muse|         modern rock|              366|
|      Mr. Brightside|      The Killers|         modern rock|              223|
|          Without Me|           Eminem|

## OneHotEncoder

In [42]:
indexer_feature = StringIndexer(inputCols=feature_cat, handleInvalid='skip', outputCols=feature_cat_indexed)
indexer_label = StringIndexer(inputCol=labelCol, handleInvalid='skip', outputCol=labelCol+'_indexed')

NameError: name 'feature_cat' is not defined

In [39]:
encoders = OneHotEncoder(dropLast=False, inputCols=feature_cat_indexed, outputCols=feature_cat_encoded)  # handleInvalid='skip',  
# sdf = encoders.fit(sdf).transform(sdf)
sdf.select(feature_cat_indexed+feature_cat_encoded).show(n=2, truncate=False, vertical=True)

NameError: name 'feature_cat_indexed' is not defined

In [35]:
pd.plotting.scatter_matrix(df)

AttributeError: 'DataFrame' object has no attribute '_get_numeric_data'